## 4SQ AND GEOJSON

In [13]:
from pymongo import MongoClient
import pandas as pd
import json
import geojson
import os

In [14]:
client = MongoClient("localhost:27017")
db = client["ironhack"]

**GEO JSON TO GET THE CITIES LOCATION AND NAIGHBOURHOODS, AND 4SQR QUERIES**

**GEO JSON**

    SAN FRANCISCO MONGO IMPORT 

In [15]:
def load_geo_json(path):

    ''' Function to load geo json.'''

    with open(path) as f:
        geo_json = json.load(f)
    return geo_json

In [16]:
san_francisco = load_geo_json("../data/SanFrancisco.Neighborhoods.json")
san_francisco["features"][0]

{'geometry': {'geometries': [{'type': 'Polygon',
    'coordinates': [[[-122.391701, 37.794113],
      [-122.39198, 37.793906],
      [-122.391614, 37.793571],
      [-122.391714, 37.793459],
      [-122.388816, 37.791005],
      [-122.388932, 37.790919],
      [-122.388616, 37.790348],
      [-122.388076, 37.790518],
      [-122.388375, 37.790334],
      [-122.388225, 37.790032],
      [-122.385852, 37.790951],
      [-122.385496, 37.790559],
      [-122.387589, 37.789838],
      [-122.387289, 37.789347],
      [-122.385303, 37.789838],
      [-122.38512, 37.789313],
      [-122.387174, 37.788807],
      [-122.387032, 37.788255],
      [-122.385261, 37.788537],
      [-122.385136, 37.788156],
      [-122.38739, 37.787736],
      [-122.387415, 37.787269],
      [-122.3845, 37.787437],
      [-122.384342, 37.785728],
      [-122.387577, 37.785485],
      [-122.38765, 37.784929],
      [-122.385546, 37.785009],
      [-122.385455, 37.784711],
      [-122.387477, 37.784493],
      [-122.38

In [17]:
def upload_geo_mongo(geojson, name_col):
    
    ''' Function to upload geo json to mongo.'''

    c = db.get_collection(name_col)

    for i in geojson["features"]:
        c.insert_one({"geo": i["geometry"]["geometries"][0], "name": i["properties"]})

In [18]:
upload_geo_mongo(san_francisco, "SanFranciscoNeigh")

    NEW YORK MONGO IMPORT 

In [19]:
new_york = load_geo_json("../data/d085e2f8d0b54d4590b1e7d1f35594c1pediacitiesnycneighborhoods.geojson")
new_york["features"][0]

{'type': 'Feature',
 'properties': {'neighborhood': 'Allerton',
  'boroughCode': '2',
  'borough': 'Bronx',
  '@id': 'http://nyc.pediacities.com/Resource/Neighborhood/Allerton'},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-73.84859700000018, 40.871670000000115],
    [-73.84582253683678, 40.870239076236174],
    [-73.85455918463374, 40.85995383576425],
    [-73.85466543306826, 40.859585694988056],
    [-73.85638870335896, 40.85759363530448],
    [-73.86888180915341, 40.857223150158326],
    [-73.86831755272824, 40.85786206225831],
    [-73.86955371467232, 40.85778409560018],
    [-73.87102485762065, 40.857309948816905],
    [-73.87048054998716, 40.865413584098484],
    [-73.87055489856489, 40.86970279858986],
    [-73.86721594442561, 40.86968966363671],
    [-73.85745, 40.86953300000018],
    [-73.85555000000011, 40.871813000000145],
    [-73.85359796757658, 40.8732883686742],
    [-73.84859700000018, 40.871670000000115]]]}}

In [20]:
def upload_geo_mongo_variable(geojson, name_col):
    
    ''' Function to upload geo json to mongo.'''

    c = db.get_collection(name_col)

    for i in geojson["features"]:
        c.insert_one({"geo": i["geometry"], "name": i["properties"]})

In [21]:
upload_geo_mongo_variable(new_york, "NewYorkNeigh")

    SEATTLE MONGO IMPORT 

In [22]:
seattle = load_geo_json("../data/Neighborhood_Map_Atlas_Neighborhoods.geojson")
seattle["features"][0]

{'type': 'Feature',
 'properties': {'OBJECTID': 27,
  'L_HOOD': 'Ballard',
  'S_HOOD': 'Loyal Heights',
  'S_HOOD_ALT_NAMES': None,
  'Shape__Area': 21320655.5455933,
  'Shape__Length': 18831.00959637},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-122.376336564723, 47.6759176989664],
    [-122.376707907517, 47.6759982290459],
    [-122.377903057631, 47.6759978849021],
    [-122.379988625303, 47.6759893429385],
    [-122.38182788443, 47.675992079457],
    [-122.383602752477, 47.675979810424],
    [-122.385435721425, 47.6759587956618],
    [-122.387087434206, 47.6759521343769],
    [-122.387702391216, 47.6759476875929],
    [-122.388955759168, 47.6759424488678],
    [-122.390449346263, 47.6759259868868],
    [-122.391773109273, 47.6759158131504],
    [-122.392827172066, 47.6759153143353],
    [-122.392956550545, 47.6759141859541],
    [-122.392956303464, 47.6763287099834],
    [-122.392959618409, 47.676929558447],
    [-122.392961263359, 47.6775085790216],
    [-122.392971502986

In [23]:
upload_geo_mongo_variable(seattle, "SeattleNeigh")